# Soluability workflow

You are given the imports below for all the necessary modules and classes required to run our example workflow

In [ ]:

import time

from ochra_discovery.storage.vessel import Vessel
from ochra_discovery.storage.reagent import Reagent
from ochra_discovery.equipment.operation import Operation
from ochra_discovery.equipment.operation_result import OperationResult
from ochra_discovery.spaces.lab import Lab
from ochra_discovery.storage.holder import Holder

from ika_rct_digital.device import IkaPlate
from tecan_xcalibur.device import TecanXCalibur
from webcam.device import Webcam
from abb_yumi.device import AbbYuMi
from xpr_quantos.device import XprQuantos
from kuka_kmriiwa.device import KukaKMRiiwa

## Connecting to the lab server
In the code box below, using `Lab` class, connect to the server at address *192.168.137.1:8001* with an experiment id of your choice.

In [ ]:
# connect to lab server
lab = Lab("192.168.137.1:8001","Ochra_Lab")

## Retrieving a station
Use `get_station` method from the `Lab` class to retrieve *yumi_station* and *kuka_station* where the latter represents KUKA KMR-iiwa mobile robot

In [ ]:
# get station
yumi_station = lab.get_station("yumi_station")
kuka_station = lab.get_station("kuka_station")

## Defining labware
In the code box below, define a rack that can hold up to 4 vials using the class `Holder`. Moreover, using the `Vessel` class, create 8 empty vials that have a maximum capacity of 15 ml

In [ ]:
# construct the rack holder
rack = Holder(type="rack", max_capacity=8)

# construct and add vials to the rack
for i in range(8):
    vial = Vessel(type="vial", max_capacity=15, capacity_unit="ml")
    rack.add_container(vial)

## Adding labware to the station
Given the rack is initially loaded into the mobile robot deck, use `add_container` method of the `inventory` attribute of our mobile robot station to add the rack to its inventory

In [ ]:
# add the rack to the station inventory
kuka_station.inventory.add_container(rack)

## Workflow steps
---

### Workflow step (1) - Load rack of vials into YuMi station
The first step in our workflow involves using the KUKA KMRiiwa mobile robot to load a rack of vials into the YuMi station so the process can begin. Use `get_robot` method of our station to retrieve our robot named *KUKA3*.

*hint: use python type hints when getting a device to have access to its methods using autocomplete*

In [ ]:
# get the mobile robot from its station station
kuka: KukaKMRiiwa = kuka_station.get_robot("KUKA3")

The mobile robot need to navigate to the YuMi station first. In the code box below, call its `go_to` method with the following arguments: {"graph_id": 1, "node_id": 30, "fine_localization": True} to do so

In [ ]:
# move the mobile robot to the yumi station
kuka.go_to({"graph_id": 1, "node_id": 30, "fine_localization": True})

Next, execute the task *PickPlaceRackDeckToStation* to load the rack of vials into the station

In [ ]:
# load the rack of vials into the station
kuka.execute("PickPlaceRackDeckToStation", {})


Before moving to the next step, we need to explicitly remove the rack of vials from the mobile robot inventory and add it to the YuMi station. To remove a rack from a station use `remove_container` method

In [ ]:
# remove the rack from the mobile robot inventory
kuka_station.inventory.remove_container(rack)

# add the rack to the yumi station inventory
yumi_station.inventory.add_container(rack)

### Workflow step (2) - Pick a vial and decap it
In this step, the YuMi robot would pick a vial from the loaded rack and decap it before moving to the next step. Use *yumi* robot from the YuMi station to pick a vial from the rack and decap it. To pick a vial use the task *pick_vial_from_rack* with the argument: {"index":1}. While to decap the a cap use the task *uncap_vial* with no arguments


In [ ]:
# get the robot from the station
yumi: AbbYuMi = yumi_station.get_robot("yumi")

# pick a vial from the rack
yumi.execute("pick_vial_from_rack", {"index":1})

# decap the vial
yumi.execute("uncap_vial", {})

### Workflow step (3) - Adding solid to the vial
We need to dispense solid into the vial using our Mettler Toledo XPR Quantos solid handler. In the code box below, do the following:
- Retrieve the solid handler named *xpr* from the station
- Open the XPR doors using `open_door` method
- Use YuMi robot to execute *load_xpr* task
- Close the XPR doors using `close_door` method
- Dispnese 20 mg of *CAFFEINE* using task named *OCHRA* with tolerance of 5
- Open the XPR doors
- Return the vial to the operation zone using the task *unload_xpr*
- Close the XPR doors

In [ ]:
# get the xpr quantos solid handler from the station
xpr: XprQuantos = yumi_station.get_device("xpr")

# open the xpr doors
xpr.open_door()

# load the xpr using yumi robot
yumi.execute("load_xpr", {})

# close the xpr doors
xpr.close_door()

# dispense 20 mg of salt with tolerance of 5
xpr.dispense(task_name="OCHRA", substance_name="CAFFEINE", amount=20, tolerance=5)

# open the xpr doors
xpr.open_door()

# return the vial to the operation zone
yumi.execute("unload_xpr", {})

# close the xpr doors
xpr.close_door()

Finally, we need to explicitly define reagent addition to the vial using `Reagent` class. In the code box, add a caffeine reagent to the first vial in the rack

In [ ]:
# retreive the vial from the rack
vial: Vessel = rack.containers[0]

# define the added reagent
salt = Reagent("CAFFEINE", 20, "mg")

# add the reagent to the vial
vial.add_reagent(salt)

### Workflow step (4) - Adding liquid to the vial
In this step, use *xcalibur_pump* liquid handler to add 10 ml of water to the vial. First, you need to load the vial into the liquid handler using the robot task *load_tecan_pump*. After finishing, use the robot task *unload_tecan_pump* to return the vial back into the operation zone and then cap it using the *cap_vial* task. Don't forget to define the reagent addition to the vial.


In [ ]:
# get the liquid handler from the station
xcalibur: TecanXCalibur = yumi_station.get_device("tecan")

# load the vial into the liquid handler
yumi.execute("load_tecan_pump", {})

# dispense 10ml of water into the vial
xcalibur.dispense(reagent="water", volume=10, unit="ml", source=2, dest=1, dir="I")

# retreive the vial from the rack
vial: Vessel = rack.containers[0]

# define the added reagent
water = Reagent("water", 10, "ml")

# add the added reagent to the vial
vial.add_reagent(water)

# unload the vial from the liquid handler
yumi.execute("unload_tecan_pump", {})

# cap the vial
yumi.execute("cap_vial", {})

### Workflow step (5) - Mixing the reagents
In order to mix the added reagents, we need to use the IKA RCT digital plate named *ika_plate* in our station. Similar to the previous steps, use the YuMi robot to load the vial into the IKA plate using the task *load_ika_plate*, stir and heat the reagents for 30 seconds, and then make the robot unload the IKA plate using the task *unload_ika_plate*

In [ ]:
# get the ika plate from the station
ika_plate: IkaPlate = yumi_station.get_device("ika_plate")

# load the vial into the ika plate
yumi.execute("load_ika_plate", {})

# start stirring
ika_plate.start_stir()

# start heating
ika_plate.start_heat()

# sleep for 10 seconds
time.sleep(10)

# stop stirring and heating
ika_plate.stop_stir()
ika_plate.stop_heat()

# unload the vial from the ika plate
yumi.execute("unload_ika_plate", {})

### Workflow step (6) - Image the resulting solution
The final step in our workflow is to take a picture of the vial to image the resulting solution before running further analysis. Like in the previous steps, first retrieve the camera device from the station using its given name *camera*. Afterwards, use the YuMi robot to present the vial to the camera using the task *goto_camera_pose*, before taking a picture using the camera apporpriate method. Finally, use the YuMi robot to return the vial to the operation zone and then to place it back in the rack using the tasks *return_from_camera_pose* and *place_vial_in_rack* respectively. The latter task takes arguments {"index":1}

*hint: store the `Operation` object returned upon calling the camera *take_image* method

In [ ]:
# get the camera from the station
camera: Webcam = yumi_station.get_device("camera")

# present the vial to the camera
yumi.execute("goto_camera_pose", {})

# take an image of the vial
op: Operation = camera.take_image()

# return the vial from the camera
yumi.execute("return_from_camera_pose", {})

# place the vial back in the rack
yumi.execute("place_vial_in_rack", {"index":1})

---
## Retrieve experiment data
After running our workflow, use the method `get_result_object` from the stored *Operation* object to retrieve `OperationResult` so we can access the taken image of the solution. Use the method `save_data` of the result object to store the image to the path /tmp/images

In [ ]:
# get the operation result object
img_result: OperationResult = op.get_result_object()

# store image to disk
img_result.save_data("img_solution")